In [1]:
import pandas as pd

In [107]:
jcust_b = pd.read_excel('../sync/prod/before/jcust.xlsx', dtype=str)
jcust_a = pd.read_excel('../sync/prod/after/jcust.xlsx', dtype=str)

In [108]:
print(len(jcust_a.columns))
print(len(jcust_b.columns))

16
16


In [ ]:
cols_b = set(jcust_b.columns.str.lower())
cols_a = set(jcust_a.columns.str.lower())

In [ ]:
only_in_b = sorted(cols_b - cols_a)
only_in_a = sorted(cols_a - cols_b)

In [ ]:
df_only_b = pd.DataFrame({'In MDG not in ECC': only_in_b})
df_only_a = pd.DataFrame({'In ECC not in MDG': only_in_a})

In [ ]:
print(df_only_a)
print(df_only_b)

Empty DataFrame
Columns: [In ECC not in MDG]
Index: []
Empty DataFrame
Columns: [In MDG not in ECC]
Index: []


In [ ]:
with pd.ExcelWriter('../sync/v-after-sync/column_mismatch_jcust.xlsx', engine='openpyxl') as writer:
    df_only_b.to_excel(writer, sheet_name='In MDG', index=False)
    df_only_a.to_excel(writer, sheet_name='In ECC', index=False)

In [ ]:
n_cols_a = list(set(jcust_a.columns) - (set(jcust_a.columns) - set(jcust_b.columns)))
n_cols_b = list(set(jcust_b.columns) - (set(jcust_b.columns) - set(jcust_a.columns)))

In [ ]:
n_cols_a.append('Supplier')
n_cols_b.append('Vendor')

In [ ]:
jcust_a = jcust_a[n_cols_a]
jcust_b = jcust_b[n_cols_b]

In [109]:
print(jcust_a.columns)
print(jcust_b.columns)

Index(['KUNNR', 'J_1IEXCD', 'J_1IEXRN', 'J_1IEXRG', 'J_1IEXDI', 'J_1IEXCO',
       'J_1ICSTNO', 'J_1ILSTNO', 'J_1IPANNO', 'J_1IEXCICU', 'AEDAT', 'USNAM',
       'J_1ISERN', 'J_1IPANREF', 'GST_TDS', 'AADHAAR_NO'],
      dtype='object')
Index(['KUNNR', 'J_1IEXCD', 'J_1IEXRN', 'J_1IEXRG', 'J_1IEXDI', 'J_1IEXCO',
       'J_1ICSTNO', 'J_1ILSTNO', 'J_1IPANNO', 'J_1IEXCICU', 'AEDAT', 'USNAM',
       'J_1ISERN', 'J_1IPANREF', 'GST_TDS', 'AADHAAR_NO'],
      dtype='object')


In [110]:
jcust_a['key'] = jcust_a.apply(
    lambda row: 
        str(row['KUNNR']),
        # + str(row['PERSNUMBER']),
        # + str(row['VTWEG'])
        # + str(row['SPART']), 
        axis=1
)

In [111]:
jcust_b['key'] = jcust_b.apply(
    lambda row: 
        str(row['KUNNR']),
        # + str(row['PERSNUMBER']),
        # + str(row['VTWEG'])
        # + str(row['SPART']), 
        axis=1
)

In [112]:
print(len(jcust_a))
print(len(jcust_b))

1593
1593


In [113]:
jcust_a.set_index('key', inplace=True)
jcust_b.set_index('key', inplace=True)

In [114]:
a_jcust_b, a_jcust_a = jcust_b.align(jcust_a, join='left')
# a_jcust_b, a_jcust_a = jcust_b, jcust_a


In [115]:
print(len(a_jcust_a))
print(len(a_jcust_b))

1593
1593


In [116]:
diffs = a_jcust_b.compare(
    a_jcust_a,
    keep_equal=True,      # show all cells
    # keep_equal=False,      # only show changed cells
    keep_shape=True,       # retain full shape if you want blanks where no change
    result_names=('Before','After')
)

In [117]:
diffs.columns = [f"{col[0]} ({col[1]})" for col in diffs.columns]

In [118]:
diffs.to_excel('../sync/prod/jcust_allData_changes.xlsx')
# diffs.to_excel('../sync/prod/jcust_changes.xlsx')

In [73]:
# Ignore

In [ ]:
f = pd.read_excel('../j1Data_changes.xlsx', sheet_name='Sheet3')

In [148]:
f.head()

,Name,email
0,3M Electro & Communication,pbhutani@mmm.com
1,3M Electro & Communication,srjanak@mmm.com
2,A P ELECTRICALS AND ENGINEERS,SVLNRAO@APEEONLINE.COM
3,A P ELECTRICALS AND ENGINEERS,VIJAY@APEEONLINE.COM
4,AAALAC INTERNATIONAL,msprouse@aaalac.org


In [153]:
def func(df):
    n = df['Name'].split()
    
    for i in n:
        if i.lower() in df['email'].lower():
            return True
    return False

In [154]:
f['new'] = f.apply(lambda row: func(row), axis=1)

In [155]:
f

,Name,email,new
0,3M Electro & Communication,pbhutani@mmm.com,False
1,3M Electro & Communication,srjanak@mmm.com,False
2,A P ELECTRICALS AND ENGINEERS,SVLNRAO@APEEONLINE.COM,True
3,A P ELECTRICALS AND ENGINEERS,VIJAY@APEEONLINE.COM,True
4,AAALAC INTERNATIONAL,msprouse@aaalac.org,True
...,...,...,...
2828,ZELLE BIOTECHNOLOGY PVT LTD,accounts@zellebiotech.com,True
2829,ZELLE BIOTECHNOLOGY PVT LTD,orders@zellebiotech.com,True
2830,ZELLE BIOTECHNOLOGY PVT LTD,service@zellebiotech.com,True
2831,ZoomInfo Technologies LLC,ar@zoominfo.com,True


In [156]:
f.to_csv('email.csv')

In [ ]:
j1cc.columns

Index(['Address Number', 'from', 'Address Version', 'To', 'Title', 'Name',
       'Name 2', 'Name 3', 'Name 4', 'Converted name (with form of address)',
       'c/o name', 'City', 'District', 'City Code', 'District.1',
       'City (Diff. from Postal City)', 'City Code.1', 'Test stat./City file',
       'Reg. Struct. Grp.', 'Postal Code', 'PO Box Postal Code',
       'Company Postal Code', 'Postl Code Extension', 'Postl Code Extension.1',
       'Postl Code Extension.2', 'PO Box', 'Undeliverable', 'PO Box w/o No.',
       'PO Box City', 'City Code.2', 'PO Box Region', 'PO box country',
       'Delivery District', 'Transportation zone', 'Street', 'Undeliverable.1',
       'Street Code', 'Street Abbreviation', 'House Number', 'Supplement',
       'House Number Range', 'Street 2', 'Street 3', 'Street 4', 'Street 5',
       'Building Code', 'Floor', 'Room Number', 'Country Key', 'Language Key',
       'Region', 'Address group', 'Flag: There are more address group assig',
       'personal a

In [ ]:
j1.apply(lambda row: '&' in str(row['Street']), axis=1).value_counts()

False    2944
True       78
Name: count, dtype: int64